## Loading Libraries

In [41]:
import pandas as pd
import numpy as np
import torch
import evaluate

## Loading Dataset

In [42]:
df = pd.read_csv('../data/processed/train_sample_processed.csv')

In [43]:
statuses = np.unique(df['OpenStatus'].values)

id2label = {idx: label for idx, label in enumerate(statuses)}
label2id = {label: idx for idx, label in enumerate(statuses)}

In [44]:

from custom_dataset import GithubDataset
from torch.utils.data import random_split
from datasets import Dataset

# Create the pytorch dataset
full_dataset = GithubDataset(df)

In [45]:

# Performing train-test split
TRAIN_DATA_SIZE = int(len(full_dataset) * 0.75)
TEST_DATA_SIZE = int(len(full_dataset) * 0.25)
train_dataset, test_dataset = random_split(full_dataset, [TRAIN_DATA_SIZE, TEST_DATA_SIZE])

train_dataset = Dataset.from_dict(train_dataset[:])
test_dataset = Dataset.from_dict(test_dataset[:])

In [46]:
train_dataset = train_dataset.rename_columns({"text_content": "text", "status": "label"})
test_dataset = test_dataset.rename_columns({"text_content": "text", "status": "label"})

columns_to_remove = [
  'tags_onehot',
  'unrecognized_tags_count',
  'reputation',
  'undeleted_answers',
  'user_life_days',
  'title'
]

train_dataset = train_dataset.remove_columns(columns_to_remove)
test_dataset = test_dataset.remove_columns(columns_to_remove)

## Experimenting With Different Model Architectures

#### COMMENT - IDEAS

We probably should:
- retrain the whole model (probably smaller) with
- better tokenizer - built up from the ground including all the names of the specific tech (languages, frameworks, IDEs, etc.)

### Setting Tokenizer

In [47]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [48]:
MAX_TEXT_CONTENT = 128

def tokenize_func(batch):
  tokenized_batch = tokenizer(batch['text'], padding=True, truncation=True, max_length=MAX_TEXT_CONTENT)
  tokenized_batch["label"] = [label2id[label] for label in batch["label"]]
  return tokenized_batch

In [49]:
tokenized_train_dataset = train_dataset.map(tokenize_func, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_func, batched=True)

Map: 100%|██████████| 35068/35068 [00:04<00:00, 7560.82 examples/s]


In [50]:
tokenized_train_dataset.column_names

['text', 'label', 'input_ids', 'attention_mask']

### Preparing Metrics

In [51]:
accuracy_metric = evaluate.load("accuracy")

In [52]:
def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=1)
  return accuracy_metric.compute(predictions=predictions, references=labels)

### Creating Models

In [53]:
from transformers import AutoModelForSequenceClassification

model_content = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=len(statuses), id2label=id2label, label2id=label2id)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Creating Trainer

In [54]:
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
  output_dir="text_content_model",
  learning_rate=2e-5,
  per_device_train_batch_size=16,
  per_device_eval_batch_size=16,
  num_train_epochs=2,
  weight_decay=0.01,
  evaluation_strategy="epoch",
  save_strategy="epoch",
  load_best_model_at_end=True,
)

trainer = Trainer(
  model=model_content,
  args=training_args,
  train_dataset=tokenized_train_dataset,
  eval_dataset=tokenized_test_dataset,
  tokenizer=tokenizer,
  data_collator=data_collator,
  compute_metrics=compute_metrics,
)

trainer.train()

  1%|          | 125/13152 [00:15<27:24,  7.92it/s]

KeyboardInterrupt: 